In [138]:
#! pip install ipython-sql 
import sqlite3
import pandas as pd
from sqlalchemy import *
from gradedb import Gradedb
from schema import *
import matplotlib.pyplot as plt

In [139]:
%load_ext sql
%sql sqlite:///gradedb.db
engine = create_engine("sqlite:///gradedb.db")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [140]:
Task_summ = text(''' SELECT 
    tk.title AS 'Assignment',
    stu.universityid,
    stu.name AS 'Student Name',
    stu.email AS 'Student Email',
    qu.title AS 'Question Title',
    qu.content AS 'Question Content',
    an.content AS 'Answer Content',
    sc.value AS 'Score total assignment'
  FROM Student stu
  LEFT JOIN Assignment asg ON asg.universityid = stu.universityid
  LEfT JOIN Task tk ON tk.taskid = asg.taskid
  LEFT JOIN Task_question tq ON tq.taskid = asg.taskid
  LEFT JOIN Question qu ON qu.questionid = tq.questionid
  LEFT JOIN Answers an ON an.questionid = tq.questionid
  LEFT JOIN Submission sb ON sb.assignmentid = asg.assignmentid
  LEFT JOIN EvaluationRequest er ON er.submissionid = sb.submissionid
  LEFT JOIN Evaluation ev ON ev.requestid = er.requestid
  LEFT JOIN EvaluationFinished ef ON ef.evaluationid = ev.evaluationid
  LEFT JOIN Scores sc ON sc.evaluationid = ev.evaluationid;
  ''')

In [141]:
df_Task_summ = pd.read_sql(Task_summ, engine)
df_Task_summ

,Assignment,universityid,Student Name,Student Email,Question Title,Question Content,Answer Content,Score total assignment
0,Task 688,S0416410,Marjorie Steele,s0416410@vuw.leidenuniv.nl,Question 1,Over brandysnap yields airframes lengthen Kenyan?,Because Leo dhole stokvel Antichrist nonaggres...,1.0
1,Task 688,S0416410,Marjorie Steele,s0416410@vuw.leidenuniv.nl,Question 1,Over brandysnap yields airframes lengthen Kenyan?,Concluding roomth œuvres majestic Port Arthur ...,1.0
2,Task 688,S0416410,Marjorie Steele,s0416410@vuw.leidenuniv.nl,Question 1,Over brandysnap yields airframes lengthen Kenyan?,Resulting in fourteen None dwarf hetter imager...,1.0
3,Task 688,S0416410,Marjorie Steele,s0416410@vuw.leidenuniv.nl,Question 1,Over brandysnap yields airframes lengthen Kenyan?,To calculate Mogadishu confers Anglo-Indian ze...,1.0
4,Task 688,S0416410,Marjorie Steele,s0416410@vuw.leidenuniv.nl,Question 2,Calculate uninterruptible seatbelts gimmer leu...,Because squabs musophobia suasively omit coron...,1.0
...,...,...,...,...,...,...,...,...
109,Task 913,S4587398,Mary Bakley,s4587398@vuw.leidenuniv.nl,Question 1,Over brandysnap yields airframes lengthen Kenyan?,Concluding roomth œuvres majestic Port Arthur ...,6.0
110,Task 913,S4587398,Mary Bakley,s4587398@vuw.leidenuniv.nl,Question 1,Over brandysnap yields airframes lengthen Kenyan?,Resulting in fourteen None dwarf hetter imager...,6.0
111,Task 913,S4587398,Mary Bakley,s4587398@vuw.leidenuniv.nl,Question 1,Over brandysnap yields airframes lengthen Kenyan?,To calculate Mogadishu confers Anglo-Indian ze...,6.0
112,Task 913,S4587398,Mary Bakley,s4587398@vuw.leidenuniv.nl,Question 6,How participials cut-and-paste underspecified ...,Given parascenium Gage repace copiously terrie...,6.0


In [142]:
def teacher_summary(Task):
    teacher_summary = (df_Task_summ[df_Task_summ['Assignment']== Task])
    df_teacher_sum = teacher_summary.drop_duplicates(subset=['Assignment','universityid','Question Title'], keep='last')
    TaskUse = df_teacher_sum.Assignment.count()
    return(df_teacher_sum)

In [143]:
df_unique = teacher_summary.drop_duplicates(subset=['Assignment','universityid','Question Title'], keep='last')
plt.hist(df_assignments['Score'], alpha=0.2,bins=20,color = "grey", label='Overall scores')
    plt.hist(df_assignments_stud['Score'], alpha=0.5,bins=20,color = "orange", label='Your score(s)')
    plt.xlim(0, 10)
    plt.legend()
    plt.show()

IndentationError: unexpected indent (1013945813.py, line 3)

In [156]:
teacher_summary('Task 405')

,Assignment,universityid,Student Name,Student Email,Question Title,Question Content,Answer Content,Score total assignment
75,Task 405,S2204697,Diana Brown,s2204697@vuw.leidenuniv.nl,Question 5,Over vibrate titubate rabbit None wainscoting?,Resulting in Kawartha Lakes Lesage kinderwhore...,NaN
76,Task 405,S2204697,Diana Brown,s2204697@vuw.leidenuniv.nl,Question 7,Over panoplies unbosoms factions unenthusiasti...,Resulting in supersolidity unbonneted troctoli...,NaN
77,Task 405,S2204697,Diana Brown,s2204697@vuw.leidenuniv.nl,Question 13,Over tribunitious high-sticked Judea Holiness ...,To calculate voyageur furies Alai Degas anther...,NaN
105,Task 405,S4587398,Mary Bakley,s4587398@vuw.leidenuniv.nl,Question 5,Over vibrate titubate rabbit None wainscoting?,Resulting in Kawartha Lakes Lesage kinderwhore...,2.0
106,Task 405,S4587398,Mary Bakley,s4587398@vuw.leidenuniv.nl,Question 7,Over panoplies unbosoms factions unenthusiasti...,Resulting in supersolidity unbonneted troctoli...,2.0
107,Task 405,S4587398,Mary Bakley,s4587398@vuw.leidenuniv.nl,Question 13,Over tribunitious high-sticked Judea Holiness ...,To calculate voyageur furies Alai Degas anther...,2.0


In [150]:
Task_summ_count = text(''' SELECT 
    tk.title AS 'Assignment',
    stu.universityid AS 'Students',
    qu.title AS 'Questions',
    an.content AS 'Answers',
    sc.value AS 'Grades given'
  FROM Student stu
  LEFT JOIN Assignment asg ON asg.universityid = stu.universityid
  LEfT JOIN Task tk ON tk.taskid = asg.taskid
  LEFT JOIN Task_question tq ON tq.taskid = asg.taskid
  LEFT JOIN Question qu ON qu.questionid = tq.questionid
  LEFT JOIN Answers an ON an.questionid = tq.questionid
  LEFT JOIN Submission sb ON sb.assignmentid = asg.assignmentid
  LEFT JOIN EvaluationRequest er ON er.submissionid = sb.submissionid
  LEFT JOIN Evaluation ev ON ev.requestid = er.requestid
  LEFT JOIN EvaluationFinished ef ON ef.evaluationid = ev.evaluationid
  LEFT JOIN Scores sc ON sc.evaluationid = ev.evaluationid;
  ''')

In [151]:
df_Task_summ_count = pd.read_sql(Task_summ_count, engine)
df_Task_summ_count

,Assignment,Students,Questions,Answers,Grades given
0,Task 688,S0225853,Question 1,Because Leo dhole stokvel Antichrist nonaggres...,5.0
1,Task 688,S0225853,Question 1,Concluding roomth œuvres majestic Port Arthur ...,5.0
2,Task 688,S0225853,Question 1,Resulting in fourteen None dwarf hetter imager...,5.0
3,Task 688,S0225853,Question 1,To calculate Mogadishu confers Anglo-Indian ze...,5.0
4,Task 688,S0225853,Question 2,Because squabs musophobia suasively omit coron...,5.0
...,...,...,...,...,...
109,Task 913,S4587398,Question 1,Concluding roomth œuvres majestic Port Arthur ...,6.0
110,Task 913,S4587398,Question 1,Resulting in fourteen None dwarf hetter imager...,6.0
111,Task 913,S4587398,Question 1,To calculate Mogadishu confers Anglo-Indian ze...,6.0
112,Task 913,S4587398,Question 6,Given parascenium Gage repace copiously terrie...,6.0


In [161]:
df_teacher_sum3 = df_Task_summ_count.drop_duplicates(subset=['Assignment','Students'], keep='last')
Ass_id_grades = df_teacher_sum3.groupby('Assignment').count()[['Students', 'Grades given']]
df_teacher_sum2 = df_Task_summ_count.drop_duplicates(subset=['Assignment','Students','Questions'], keep='last')
Ass_question_answer = df_teacher_sum2.groupby('Assignment').count()[['Questions', 'Answers']]
Ass_question_answer['Unaswerd questions'] = Ass_question_answer['Questions'] - Ass_question_answer['Answers']
horizontal_stack = pd.concat([Ass_question_answer, Ass_id_grades], axis=1)
horizontal_stack['Missing Grades'] = horizontal_stack['Students'] - horizontal_stack['Grades given']
horizontal_stack

,Questions,Answers,Unaswerd questions,Students,Grades given,Missing Grades
Assignment,,,,,,
Task 405,6,6,0,2,1,1
Task 688,12,12,0,4,4,0
Task 871,12,12,0,4,3,1
Task 913,9,6,3,3,2,1
Task 980,6,4,2,2,1,1
